In [ ]:
import pandas as pd
import numpy as np
import plotly.express as ex
import re

In [ ]:
def get_cal_value(fn, show_plot=True, units='lb'):
    d = pd.read_csv(fn,names=['adc'])
    get_mass = re.compile('_(\d+\.*\d+)_lb')
    mass = float(get_mass.findall(fn)[0])

    dy    = np.diff(d.adc)
    y_min = min(d.adc) * 1.2
    y_max = max(d.adc) * 1.2

    # print(f"[{y_min}, {y_max}]")
    ix = [np.argmin(dy)+2, np.argmax(dy)-2]
    ym = [min(d.adc), min(d.adc)]
    
    ii = (d.index > ix[0]) & (d.index < ix[1])
    low = np.mean(d.adc[ii]) 
    
    ix = [np.argmin(dy)-2, np.argmax(dy)+2]
    ii = (d.index > ix[0]) & (d.index < ix[1])
    bck = np.mean(d.adc[~ii]) 
    if show_plot:
        fig = ex.line(x=d.index, y=d.adc,  height=600)
        # fig.add_scatter(x=[ix[0], ix[0]], y=[y_min, y_max])
        # fig.add_scatter(x=[ix[1], ix[1]], y=[y_min, y_max])

        fig.add_scatter(x=[d.index.start, d.index.stop], y=[low, low])    
        fig.add_scatter(x=[d.index.start, d.index.stop], y=[bck, bck])
        fig.show()

    if units == 'lb':
        mass_kg = (mass*0.453)
    else:
        mass_kg = (mass/1000)

    last_cal = round((y_max - y_min)/mass_kg)    
    print(f"Mass {mass_kg:.3f} kg = {last_cal} LSB")
    return [mass_kg, (y_max - y_min), y_min, last_cal]

In [ ]:
import glob as g
CAL_LSB_PER_KG = 25530.5

files = g.glob(f'cal/cal_*_lb.csv')
full_cal = []
for fn in files:
    full_cal.append(get_cal_value(fn, show_plot=False, units='lb'))

In [ ]:
shp = np.shape(full_cal)
x = []
y = []
for r in full_cal:
    x.append(abs(r[2]))
    y.append(r[0])

In [ ]:
import copy 
fit = np.polyfit(x,y,1)
x_fit = copy.copy(x)
y_fit = np.polyval(fit, x_fit)
fig = ex.line(x=x_fit, y=np.polyval(fit,x_fit),  height=600)
fig.add_scatter(x=x, y=y);
fig.show()

In [ ]:
fit = np.array([ 3.86573970e-05, -1.55586835e+01])
d = pd.read_csv('catdata_03-17.csv',names=['timestamp','adc','temp'])
t = d.timestamp
t = (t - t[0]) / 3600
y = np.polyval(fit, abs(d.adc))

d['weight'] = y
d['t'] = t
fig = ex.line(x=t, y=y, labels=dict(x="Time [h]", y="Mass [kg]"))
fig.show()

In [ ]:
y = y - min(y)
fig = ex.line(x=t[0:-1], y=abs(np.diff(y)))
fig.show()

In [ ]:
ix = (d.t <24) & (d.temp < 85) & (d.weight < 10)
ii = ix.values
# temp = d.temp
tt   = d.t.iloc[ii]
y    = d.weight[ii]
tmp  = d.temp[ii]
fig = ex.line(x=tt, y=tmp)
fig.show()
fig = ex.line(x=tt, y=-1*y)
fig.show()
fig = ex.scatter(x=tmp, y=y)
fig.show()